# Factorwise Averages

In [ ]:
import numpy as np

In [ ]:
def factorwise_matrix(
    mat: "Mat to be factored, just a vector of the diagonals",
    ds: "Dimensions of factor matrices",
    idx: "Indices",
    k: "Dimension"
):
    """
    A(i, j | k) notation from TeraLasso
    """
    
    # Stride-blocking based off of StackOverflow answer:
    # https://stackoverflow.com/a/8070716/10642078
    d = ds[k]
    d_left = np.prod(ds[:k]).astype(int)
    d_right = np.prod(ds[k+1:]).astype(int)
    size = d_left * d * d_right
    
    sz = a.itemsize
    shape = (size//d_left, size//d_left, d_left, d_left)

    strides = sz * np.array([
        w * d_left,
        d_left,
        w,
        1
    ])
    blocks = np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)
    
    # Now that we've got the strides, we need to pick the right one based on the idxs
    i, j = idx
    
    # Last two dimensions are the sizes of the blocks
    specific_block =  blocks[i::d, j::d]
    
    # Now we need to de-stride the block
    return np.concatenate(
        np.concatenate(
            specific_block,
            axis=1
        ),
        axis=1
    )
    
n=8
m=8
a = np.arange(1,n*m+1).reshape(n,m)
print(a)
out = factorwise_matrix(a, [2, 2, 2], (0, 0), 1)

out

[[ 1  2  3  4  5  6  7  8]
 [ 9 10 11 12 13 14 15 16]
 [17 18 19 20 21 22 23 24]
 [25 26 27 28 29 30 31 32]
 [33 34 35 36 37 38 39 40]
 [41 42 43 44 45 46 47 48]
 [49 50 51 52 53 54 55 56]
 [57 58 59 60 61 62 63 64]]


array([[ 1,  2,  5,  6],
       [ 9, 10, 13, 14],
       [33, 34, 37, 38],
       [41, 42, 45, 46]])

In [ ]:
def factorwise_average(
    mat: "Mat to be factored, just a vector of the diagonals",
    ds: "Dimensions of factor matrices",
    k: "Dimension"
):
    d = ds[k]
    d_left = np.prod(ds[:k]).astype(int)
    d_right = np.prod(ds[k+1:]).astype(int)
    d_non = d_left * d_right
    out = np.zeros((d_non, d_non))
    for i in range(d):
        out += factorwise_matrix(mat, ds, (i, i), k)
    return out / d_non
factorwise_average(a, [2, 2, 2], 1)

array([[ 5. ,  5.5,  7. ,  7.5],
       [ 9. ,  9.5, 11. , 11.5],
       [21. , 21.5, 23. , 23.5],
       [25. , 25.5, 27. , 27.5]])

In [ ]:
def kronecker_factor(
    mat: "Mat to be factored, just a vector of the diagonals",
    ds: "Dimensions of factor matrices",
    k: "Dimension"
):
    K = len(ds)
    A = factorwise_average(mat, ds, k)
    offset = (K-1)/K * np.trace(A) / ds[k]
    return A - offset * np.eye(A.shape[0])

kronecker_factor(a, [2, 2, 2], 1)

array([[-16.66666667,   5.5       ,   7.        ,   7.5       ],
       [  9.        , -12.16666667,  11.        ,  11.5       ],
       [ 21.        ,  21.5       ,   1.33333333,  23.5       ],
       [ 25.        ,  25.5       ,  27.        ,   5.83333333]])

In [ ]:
def kron_sum(A, B):
    """
    Computes the kronecker sum of two square input matrices
    
    Note: `scipy.sparse.kronsum` is a thing that would
    be useful - but it seems that `scipy.sparse` is not
    yet a mature library to use.
    """
    a, _ = A.shape
    b, _ = B.shape
    return np.kron(A, np.eye(b)) + np.kron(np.eye(a), B)

In [ ]:
# This is wrong, but why?
factor = np.arange(9).reshape(3, 3) - 4
b = kron_sum(factor, factor)
print(factor)
print(b)
kronecker_factor(b, [3, 3], 0)

[[-4 -3 -2]
 [-1  0  1]
 [ 2  3  4]]
[[-8. -3. -2. -3. -0. -0. -2. -0. -0.]
 [-1. -4.  1. -0. -3.  0. -0. -2.  0.]
 [ 2.  3.  0.  0.  0. -3.  0.  0. -2.]
 [-1. -0. -0. -4. -3. -2.  1.  0.  0.]
 [-0. -1.  0. -1.  0.  1.  0.  1.  0.]
 [ 0.  0. -1.  2.  3.  4.  0.  0.  1.]
 [ 2.  0.  0.  3.  0.  0.  0. -3. -2.]
 [ 0.  2.  0.  0.  3.  0. -1.  4.  1.]
 [ 0.  0.  2.  0.  0.  3.  2.  3.  8.]]


array([[-4.94444444, 13.        , 16.        ],
       [34.        , 22.05555556, 40.        ],
       [36.66666667, 39.66666667, 27.72222222]])